In [16]:
%matplotlib notebook
import matplotlib
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal, interpolate,fftpack
import csv

In [31]:
data_dir = '../datasets/slipperData/'
filter_dir = '../datasets/filteredData/'
data_name = 'Alice.csv'
header = pd.read_csv(data_dir + data_name, nrows=1, header=None)
header = header.values[0]

In [32]:
#alice = pd.read_csv('Project/slipperData/Alice.csv', nrows=5000,skiprows=400,names=header)
alice = pd.read_csv('../datasets/slipperData/Alice.csv', skiprows=1, names=header)

In [33]:
ts = alice['Timestamp'].values
ts_orig = ts - min(ts)
axis = [alice['Axis' + str(i+1)].values for i in range(3)]

In [34]:
fs = 33
ps = 1/fs
n = np.int16(8192)
freq = np.linspace(0, fs/2, num=int(n//2))
ts_new =  np.arange(min(ts_orig), max(ts_orig), ps)
axis_interpolant = [interpolate.interp1d(ts_orig, axis[i]) for i in range(3) ]
axis_new = [axis_interpolant[i](ts_new) for i in range(3) ]

In [35]:
axis_new = [axis_new[i] - np.mean(axis_new[i]) for i in range(3)]

In [70]:
filter_length = 5
win = np.ones(filter_length)
i=1
axis_lpf = [signal.convolve(axis_new[i], win, mode='same') / filter_length for i in range(3)]

import csv
with open(filter_dir + data_name, 'w') as csvfile:
    csvw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    csvw.writerow(["Time", "Axis 1", "Axis 2", "Axis 3"])
    for i in range(len(ts_new)):
        csvw.writerow([ts_new[i], axis_lpf[0][i], axis_lpf[1][i], axis_lpf[2][i]])
        
plt.plot(ts_new, axis_lpf[0])
plt.xlim([10,20])


(10, 20)